In [2]:
import matplotlib.pyplot as plt
from ultralytics import YOLO
from mmcv.image import imread
from mmpose.apis import inference_topdown, init_model
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples
from tqdm import tqdm
import os, json, glob, shutil, cv2
import copy
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

#labeling
center = ["U1","U2","U3","U4","U5","U6","U7","U8","D1","D2","D3","D4","D5","D6","D7","D8","D9"]

# YOLO 모델 로드
yolo_model = YOLO('/RnD/dog_yolo/results/train7/weights/best.pt')  # custom한 YOLOv8 모델 사용

#rtmpose 모델 로드
pose_config = '/RnD/mmpose/dog_mmpose/Test/rtmpose_s_8xb256_420e_aic_coco_256x192.py'
pose_checkpoint = '/RnD/mmpose/dog_mmpose/Test/best_coco_AP_epoch_300.pth'
device = 'cpu'
rtm_model = init_model(pose_config, pose_checkpoint, device=device)


# 디렉토리 설정
image_dir = '/RnD/TRIGHT_REST/'  # 이미지
output_dir = '/RnD/mmpose/dog_mmpose/mm+yolo/'  # rtmpose 결과 저장

# 출력 디렉토리가 존재하지 않으면 생성
os.makedirs(output_dir, exist_ok=True)

# 지원할 이미지 파일 확장자
image_extensions = ['*.jpg', '*.jpeg', '*.png']

# 모든 이미지 파일 경로 가져오기
image_paths = []
for ext in image_extensions:
    image_paths.extend(glob.glob(os.path.join(image_dir, ext)))

# (5) 필터링 된 이미지 파일 처리 및 바운딩 박스 정보 저장
for img_path in image_paths:
    # 이미지 로드
    image = cv2.imread(img_path)
    if image is None:
        print(f"Image not found: {img_path}")
        continue
    height,width,_=image.shape #이미지 height,width 측정 
    # 객체 탐지
    results = yolo_model(image)
    #1. 바운딩 박스 정보 저장
    for result in results:
        boxes = result.boxes  # 바운딩 박스 정보
        #print(boxes)
        x1, y1 = int(0.25 * width), int(0.25 * height) # undifined방지> 바운딩 박스 기본값 설정 (이미지 중앙 부분)
        x2, y2 = int(0.75 * width), int(0.75 * height)
        boxcal_data=[[x1,y1],[x2,y2]] #bbox
        if boxes is None: 
            continue
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            boxcal_data=[[x1,y1],[x2,y2]] #bbox
            #2. 바운딩 박스 확장 (좌우 5%씩)
            box_width = x2 - x1
            box_height = y2 - y1
            x1 = int(x1 - 0.05 * width)
            y1 = int(y1 - 0.05 * height)
            x2 = int(x2 + 0.05 * width)
            y2 = int(y2 + 0.05 * height)

            # 이미지 범위를 벗어나지 않도록 조정
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(image.shape[1], x2)
            y2 = min(image.shape[0], y2)

    #print(x1, y1, x2, y2)

    #3. 이미지 크롭하기
    cropped_image = image[y1:y2, x1:x2]

    #4. 이미지 리사이즈 256X192
    rtmpose_size=(256,192)
    cropped_image=cv2.resize(cropped_image,rtmpose_size)

    #크롭한 이미지 비주얼화
    # plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
    # plt.title('Cropped Image')
    # plt.show()

    # 5. rtmpose 적용하기 -keypoint 가져오기 
    batch_results=inference_topdown(rtm_model,cropped_image)
    #print(batch_results[0].pred_instances.keypoints_visible)
    #min-max 정규화->데이터최대최소를 설정해서 그안에서 상대적인 대소관계유지 
    kpntscores=batch_results[0].pred_instances.keypoints_visible
    min_num=min(kpntscores[0])
    max_num=max(kpntscores[0])
    normalized_scores=(kpntscores[0]-min_num)/(max_num-min_num)
    #print(normalized_scores)
    # 키포인트를 크롭된 이미지에 찍기
    keypnt=batch_results[0].pred_instances.keypoints[0]
    keypnt_data=keypnt.tolist()

    #cnn 모델 ------------------------------------ 
    # 좌표 색상(픽셀값) 확인하기
    def classify_color(color):
        R, G, B = color[2], color[1], color[0]
        
        if 200 <= R <= 255 and 200 <= G <= 255 and 200 <= B <= 255:
            return 'White'
        elif 150 <= R <= 255 and 50 <= G <= 200 and 150 <= B <= 255:
            return 'Pink'
        elif 150 <= R <= 255 and 0 <= G <= 100 and 0 <= B <= 100:
            return 'Red'
        elif 80 <= R <= 200 and 40 <= G <= 150 and 0 <= B <= 100:
            return 'Brown'
        else: #그 외의 색상이나 이미지가 범위에 들어가지 않는경우 일괄로 others로 설정 
            return 'Other'
    
    #픽셀 값: 각 픽셀의 색상과 밝기 정의 (RGB에서는 각 픽셀은 세가지 색상 채널의 값을 가짐 red,green,blue)
    label_color=[]
    for i in range(17):
        x, y = int(keypnt_data[i][0]), int(keypnt_data[i][1]) #정수화->특정 좌표의 픽셀 값을 가져올 때 정수형 좌표가 필요(소수 안됨)
        if 0 <= x < cropped_image.shape[1] and 0 <= y < cropped_image.shape[0]:  # 좌표가 이미지 범위 내에 있는지 확인
            color = cropped_image[y, x]
            label_color.append(classify_color(color))
        else:
            label_color.append('Other')
    # print(label_color)

    #픽셀값과 임계값에 따라서 visible 결정하기 
    label_id=[]
    for i in range(17): #라벨링 0<->1이나 1<->2는 있지만 0<->2인 경우는 없으므로 따로 만들지 않음  
        if i==3 or i==11: # u4 d4는 2로
            #1 2 3 4 5 6 7 8 1 2 3 4 5 6 7 8 9
            label_id.append(2)
        elif normalized_scores[i]>=0.6 : #2
            label_id.append(2)
        elif normalized_scores[i]>=0.3 : #1
            if label_color[i]=='Pink' or label_color[i]=='Red' or label_color[i]=='Brown': #1인테 핑크나 갈색인경우 0으로 (입술이나 잇몸)
                #print('change value 1 to 0 at',i)
                label_id.append(0) 
                continue
            label_id.append(1)
        else: #0
            if label_color[i]=='White':#0인데 흰색인 경우 (치아일 가능성이 있음)
                #print('change value 0 to 1 at',i)
                label_id.append(1)
                continue
            label_id.append(0)
                  

    # 크롭된 이미지의 키포인트를 원본 이미지의 좌표계로 변환
    scale_x = (x2 - x1) / rtmpose_size[0]
    scale_y = (y2 - y1) / rtmpose_size[1]
    keypoints_original = np.array(keypnt_data) * [scale_x, scale_y] + [x1, y1]
    
    # 원본 이미지에 키포인트 시각화
    for point in keypoints_original:
        cv2.circle(image, (int(point[0]), int(point[1])), 5, (0, 0, 255), -1)
    #키포인트를 적용한 크롭된 이미지 시각화
    cv2.rectangle(image,(boxcal_data[0][0],boxcal_data[0][1]),(boxcal_data[1][0],boxcal_data[1][1]),color=(255,0,0), thickness=2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()
    
    # JSON 데이터 구조
    new_data = {
        "version": "0.3.3",
        "flags": {},
        "shapes": [],
        "imagePath": os.path.basename(img_path),
        "imageData": None,
        "imageHeight": height,
        "imageWidth": width,
        "text": ""
    }
    shapes = {
        "label": "",
        "text": "",
        "points": [],
        "group_id": 2,
        "shape_type": "point",
        "flags": {}
    }
    for i in range(18):
        shape = copy.deepcopy(shapes)
        new_data['shapes'].append(shape)
    # center 키포인트
    for i in range(17):
        new_data['shapes'][i]['points'] = [keypoints_original[i].tolist()]
        new_data['shapes'][i]['label'] = center[i]
        new_data['shapes'][i]['group_id']=label_id[i]
    # tooth rectangle
    new_data['shapes'][17]['points'] = boxcal_data
    new_data['shapes'][17]['label'] = 'dog_side'
    new_data['shapes'][17]['shape_type'] = 'rectangle'
    if new_data['shapes'][17]['label']== 'dog_side':
        new_data['shapes'][17]['group_id'] = None
    
    json_data = json.dumps(new_data, indent=4)
    json_file_name = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(img_path))[0]}.json")
    with open(json_file_name, 'w') as json_file:
        json_file.write(json_data)
    #print(json_data)
    # print(f"Saved JSON for image:{json_file_name}")

ImportError: /opt/conda/lib/python3.10/site-packages/mmcv/_ext.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZNK3c106SymIntltEl